In [1]:
from datetime import datetime
from json import loads
import logging
from urllib import parse

import requests

logging.basicConfig(format="%(asctime)s [%(levelname)8s]: %(message)s [%(name)s]",
                        level=logging.INFO)
logger = logging.getLogger(__name__)
logger.info("Logging ready")

2021-04-21 00:11:28,345 [    INFO]: Logging ready [__main__]


In [2]:
# campground_id = 232493 # FISH CREEK
# campground_id = 232463 # RMNP
campground_id = 232462 # RMNP, GLACIER BASIN
campground_id

232462

In [7]:
search_date = datetime(year=2021, month=8, day=21)
year = search_date.year
month = str(search_date.month).zfill(2)
start_date = f"{year}-{month}-01T00:00:00.000Z"
query_string = parse.urlencode(query=dict(start_date=start_date))

url_components = dict(scheme="https",
                      netloc="recreation.gov",
                      url=f"api/camps/availability/campground/{campground_id}/month",
                      params="", query=query_string, fragment="")
api_endpoint = parse.urlunparse(tuple(url_components.values()))
api_endpoint

'https://recreation.gov/api/camps/availability/campground/232462/month?start_date=2021-08-01T00%3A00%3A00.000Z'

In [4]:
web_browser_headers = {'User-Agent':
                           'Mozilla/5.0 (X11; Linux x86_64; rv:10.0) Gecko/20100101 Firefox/10.0'}
response = requests.get(url=api_endpoint,
                        headers=web_browser_headers)
# display(HTML(response.text))
data = loads(response.content)
response

<Response [200]>

In [5]:
unavailable_strings: list = [
    "Reserved",
    "Not Available",
    "Not Reservable Management"
]


In [6]:
campsites_string = "campsites"
data_copy = data.copy()
campsite_data_copy = data_copy[campsites_string]

for campsite_id, campsite_data in campsite_data_copy.items():
    campsite_data_availabilities = campsite_data["availabilities"]
    campsite_loop = campsite_data.get("loop", "Default Loop")
    site_name = campsite_data.get("site", "Default Site")
    campsite_location = " - ".join([campsite_loop, site_name])
    for date_string, availability_status in campsite_data_availabilities.items():
        matching_date = datetime.strptime(date_string, "%Y-%m-%dT%H:%M:%SZ")
        if availability_status not in unavailable_strings:
            print(campsite_id, campsite_location, matching_date, availability_status)

1434 loop C - C098 2021-08-26 00:00:00 Available
1526 Loop B - B036 2021-08-31 00:00:00 Available
1531 Loop B - B042 2021-08-31 00:00:00 Available
1539 Loop B - B049 2021-08-31 00:00:00 Available
1541 Loop B - B051 2021-08-31 00:00:00 Available
1544 loop C - C065 2021-08-31 00:00:00 Available
1557 loop C - C068 2021-08-26 00:00:00 Available
1560 loop C - C071 2021-08-31 00:00:00 Available
1565 loop C - C076 2021-08-26 00:00:00 Available
1573 loop C - C085 2021-08-31 00:00:00 Available
